## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
import warnings

pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

### Pass API Key

In [2]:
c = Census("e0577a26a616f4dda60446eae987e3b6d0d944a3")

### Preview Variable

In [3]:
cs.printtable(cs.censustable('acs5',2019,'B08301'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08301_001E  | MEANS OF TRANSPORTATION TO WOR | !! Estimate Total:                                       | int  
B08301_002E  | MEANS OF TRANSPORTATION TO WOR | !! !! Estimate Total: Car, truck, or van:                | int  
B08301_003E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! Estimate Total: Car, truck, or van: Drove alone | int  
B08301_004E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! Estimate Total: Car, truck, or van: Carpooled:  | int  
B08301_005E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or van: Carpoole | int  
B08301_006E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or van: Carpoole | int  
B08301_007E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or va

### Get Data

- **ACS 1 Year**

In [4]:
Years = [2019,2023]
AllData = []

for year in Years:
    

    ACS = c.acs1.get(
        ["B08301_001E",'B08301_003E','B08301_004E','B08301_010E','B08301_019E','B08301_018E','B08301_016E',
        'B08301_017E','B08301_020E','B08301_021E'],
        {'for': 'place:04000', 'in': 'state:48'}, year  = year)

    Data = pd.DataFrame(ACS)
    
    Data.rename(columns={'B08301_001E': 'Total','B08301_003E': 'Drove Alone','B08301_004E': 'Carpool',
                          'B08301_010E': 'Public Transit','B08301_019E': 'Walked','B08301_018E': 'Bicycle',
                          'B08301_017E': 'Motorcycle','B08301_020E': 'Other','B08301_021E': 'Work From Home',
                          'B08301_016E': 'Taxicab'},  inplace = True)
    
    CP = ['Drove Alone', 'Carpool', 'Public Transit', 'Walked','Bicycle', 'Taxicab', 
          'Motorcycle', 'Other', 'Work From Home']
   

    for row in CP:
        Data[row] = (Data[row].astype(float) / Data['Total'].astype(float)) * 100  # Calculate Percentage
        
        
    Data = pd.DataFrame(Data[CP])

    Data.to_csv(f'Commuting Patterns {year} ACS 1 year Estimate.csv',index = False) #Export Data

    DataLong = Data.melt(id_vars=[], value_vars=CP,var_name='Commute Mode', value_name= 'Percentage')  # convert to Long
    DataLong['Year'] = year

    AllData.append(DataLong)
        

Datacombined = pd.concat(AllData) # combine all

Datacombined['Year'] = Datacombined['Year'].astype(str) 

DataPivot = Datacombined.pivot(index='Commute Mode', columns='Year', values='Percentage').reset_index() # Make year columns


DataPivot['Percentage Points Difference (2019-2023)'] = DataPivot['2023'] - DataPivot['2019']

DataPivot['Percentage Change (2019-2023)'] = ((DataPivot['2023'] - DataPivot['2019']) / DataPivot['2019']) * 100

DataPivot = DataPivot.round(2)

DataPivot.to_csv(f'Commuting Patterns 2019-2023 ACS 1-year City of Arlington.csv', index=False)

- **ACS 5 Year**

In [5]:
Years = [2019,2023]
AllData = []

for year in Years:
    

    ACS = c.acs5.get(
        ["B08301_001E",'B08301_003E','B08301_004E','B08301_010E','B08301_019E','B08301_018E','B08301_016E',
        'B08301_017E','B08301_020E','B08301_021E'],
        {'for': 'place:04000', 'in': 'state:48'}, year  = year)

    Data = pd.DataFrame(ACS)
    
    Data.rename(columns={'B08301_001E': 'Total','B08301_003E': 'Drove Alone','B08301_004E': 'Carpool',
                          'B08301_010E': 'Public Transit','B08301_019E': 'Walked','B08301_018E': 'Bicycle',
                          'B08301_017E': 'Motorcycle','B08301_020E': 'Other','B08301_021E': 'Work From Home',
                          'B08301_016E': 'Taxicab'},  inplace = True)
    
    CP = ['Drove Alone', 'Carpool', 'Public Transit', 'Walked','Bicycle', 'Taxicab', 
          'Motorcycle', 'Other', 'Work From Home']
   

    for row in CP:
        Data[row] = (Data[row].astype(float) / Data['Total'].astype(float)) * 100  # Calculate Percentage
        
        
    Data = pd.DataFrame(Data[CP])

    Data.to_csv(f'Commuting Patterns {year} ACS 5-year Estimate.csv',index = False) #Export Data

    DataLong = Data.melt(id_vars=[], value_vars=CP,var_name='Commute Mode', value_name= 'Percentage')  # convert to Long
    DataLong['Year'] = year

    AllData.append(DataLong)
        

Datacombined = pd.concat(AllData) # combine all

Datacombined['Year'] = Datacombined['Year'].astype(str) 

DataPivot = Datacombined.pivot(index='Commute Mode', columns='Year', values='Percentage').reset_index() # Make year columns


DataPivot['Percentage Points Difference (2019-2023)'] = DataPivot['2023'] - DataPivot['2019']

DataPivot['Percentage Change (2019-2023)'] = ((DataPivot['2023'] - DataPivot['2019']) / DataPivot['2019']) * 100

DataPivot = DataPivot.round(2)

DataPivot.to_csv(f'Commuting Patterns 2019-2023 ACS 5-year City of Arlington.csv', index=False)